## Idea:
Given $k$, build a model to predict the number of visitors after k days using the following features:
1. (holidayflag, day_of_week, is_closed, #visitors) for the past two weeks.
2. store_id, gentre, area

Only do label encoding to categorical variables

In [1]:
import os
import numpy as np
import pandas as pd
import platform
import pickle
from pathlib import Path
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%qtconsole

In [3]:
if platform.system() == 'Linux':
    data_dir = '/home/alin/Data/Recruit_Holding'
else:
    data_dir = 'C:/Users/alin/Documents/Data/Recruit_Holding'

In [4]:
DFS_dump = data_dir + '/DFS.p'
if Path(DFS_dump).is_file():
    print('load previous dump')
    DFS = pickle.load(open(DFS_dump, 'rb'))
    air_reserve = DFS['air_reserve']
    air_reserve_day = DFS['air_reserve_day']
    hpg_reserve = DFS['hpg_reserve']
    hpg_reserve_day = DFS['hpg_reserve_day']
    air_visit_hist = DFS['air_visit_hist']
    date_info = DFS['date_info']
    test = DFS['test']
    air_store_info = DFS['air_store_info']
    hpg_store_info = DFS['hpg_store_info']
    store_id_relation = DFS['store_id_relation']
    test = DFS['test']
else:
    print('run EDA1 first')

load previous dump


### Build the training and testing datasets before label encoding

### step 1: add dates when a store is closed.

In [43]:
from itertools import product

In [32]:
k = 3
last_train_day = max(air_visit_hist.day_ind)
first_train_day = last_train_day - k * 14 + 1

In [36]:
hist1 = air_visit_hist[(air_visit_hist.day_ind >= first_train_day) & (air_visit_hist.day_ind <= last_train_day)].copy()

In [38]:
all_stores = hist1.air_store_id.unique()

In [40]:
all_days = [i for i in range(first_train_day, last_train_day+1)]

In [58]:
grid = np.array(list(product(*[all_stores, all_days])))

In [60]:
grid = pd.DataFrame(grid, columns=['air_store_id', 'day_ind_str' ])

In [63]:
grid['day_ind'] = grid.apply(lambda r: int(r['day_ind_str']), axis=1)

In [66]:
grid.drop('day_ind_str', axis=1, inplace=True)

In [101]:
all_data = grid.merge(hist1, how='left', on=['air_store_id', 'day_ind'])

In [102]:
all_data = all_data.merge(date_info, on='day_ind', suffixes=['_l', ''])

In [103]:
drop_columns = [col for col in all_data.columns if col[-1] == 'l']

In [104]:
all_data.drop(drop_columns, inplace=True, axis=1)

In [105]:
all_data = all_data.merge(air_store_info, on = 'air_store_id', suffixes = ['_l', ''])

In [106]:
drop_columns = [col for col in all_data.columns if col[-1] == 'l'] + ['calendar_date', 'date', 'latitude', 'longitude', 'hpg_store_id']

In [107]:
all_data.drop(drop_columns, inplace=True, axis=1)

In [112]:
all_data['closed'] = all_data.apply(lambda r: 1 if pd.isnull(r['visitors']) else 0, axis=1)

In [116]:
all_data.fillna(0, inplace=True)